# Configuration

In [ ]:
# Parameters
PROJECT_NAME = 'ML1010-Group-Project'
ENABLE_COLAB = True

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni/Documents/ML_Projects'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

# Bootstrap Environment

In [ ]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)



import mv_python_utils as mvutils


Mounted at /content/gdrive
Wha...where am I?
I am awake now.

I have set your current working directory to /content/gdrive/MyDrive/Colab Notebooks/ML1010-Group-Project
The current time is 17:36
Hello sir. Reminder, no more coffee.



# Setup Runtime Environment

In [ ]:
#%%capture
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q 
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
#import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt

!python -m spacy download en_core_web_sm

nltk.download('stopwords')
%matplotlib inline

import cw_df_metric_utils as cwutils

     |████████████████████████████████| 981 kB 6.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 64 kB 3.2 MB/s 
     |████████████████████████████████| 62 kB 965 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 6.0 MB 9.9 MB/s 
     |████████████████████████████████| 628 kB 80.9 MB/s 
     |████████████████████████████████| 10.1 MB 56.5 MB/s 
     |█████████████

'Google Colab enabled'

     |████████████████████████████████| 13.9 MB 565 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Data

In [ ]:
jarvis.showProjectDataFiles()

Here are all your project data files
[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project [Empty directory]

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/01_original
---[   gz][ json]--> Cell_Phones_and_Accessories_5.json.gz (161.24 MB)
---[   gz][ json]--> meta_Cell_Phones_and_Accessories.json.gz (343.33 MB)

[D] /content/gdrive/MyDrive/Colab Notebooks/data/ML1010-Group-Project/02_working
[*][  pkl]---------> 01_Cellphone_small.pkl (45.46 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_1.pkl.gz (6.88 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_2.pkl.gz (170.55 MB)
---[   gz][  pkl]--> 01_NLP_ReviewText_Narrow_3.pkl.gz (295.59 MB)
[*][  pkl]---------> 01_NLP_ReviewText_small.pkl (28.94 MB)
[*][  pkl]---------> 01_NLP_Summary_small.pkl (3.82 MB)
[*][  pkl]---------> 01_NLP_Title_small.pkl (2.73 MB)
---[   gz][  pkl]--> 01_NL_ReviewText_All.pkl.gz (592.92 MB)
---[   gz][  pkl]--> 01_NL_ReviewText_textSplit.pkl.gz (15.78 MB)
[*][  pkl]----

In [ ]:
### load the meta data
dataOrig = pd.read_pickle(jarvis.DATA_DIR_WORK + "/01_NL_ReviewText_All.pkl.gz")
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   uuid                             63413 non-null  object 
 1   reviewText                       63413 non-null  object 
 2   overall                          63413 non-null  float64
 3   reviewText_lemma                 63413 non-null  object 
 4   reviewText_nouns                 63413 non-null  object 
 5   reviewText_adjectives            63413 non-null  object 
 6   reviewText_verbs                 63413 non-null  object 
 7   reviewText_nav                   63413 non-null  object 
 8   reviewText_lemma_tb_pol          63310 non-null  float64
 9   reviewText_lemma_tb_subj         63310 non-null  float64
 10  reviewText_lemma_tb_tokens       63310 non-null  float64
 11  reviewText_lemma_tb_length       63310 non-null  float64
 12  reviewText_lemma_b

# Column Explosion

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
%%time
#Step 6 - Explode NLP text columns
if ENABLE_COLAB: 
  #cwutils.expandColumn(dataOrig, 'reviewText')
  
  #cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_lemma')
  #cwutils.bertEncode(dataOrig, 'reviewText_lemma')
  #cwutils.flairSentimentEncode(dataOrig, 'reviewText_lemma')
  
  #cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_adjectives')
  #cwutils.bertEncode(dataOrig, 'reviewText_adjectives')
  #cwutils.flairSentimentEncode(dataOrig, 'reviewText_adjectives')
  
  #cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_nav')
  #cwutils.bertEncode(dataOrig, 'reviewText_nav')
  #cwutils.flairSentimentEncode(dataOrig, 'reviewText_nav')
  
  #cwutils.splitSpacySentences(dataOrig, 'reviewText_lemma')  
  display("Colab work done")
else:
  cwutils.expandColumn(dataOrig, 'reviewText')
  cwutils.flairSentimentEncode(dataOrig, 'reviewText_lemma')
  cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_lemma')
  cwutils.bertEncode(dataOrig, 'reviewText_lemma')
  cwutils.splitSpacySentences(dataOrig, 'reviewText_lemma')  

'Colab work done'

CPU times: user 2.57 ms, sys: 289 µs, total: 2.86 ms
Wall time: 2.7 ms


# Scratchpad

In [ ]:
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   uuid                             63413 non-null  object 
 1   reviewText                       63413 non-null  object 
 2   overall                          63413 non-null  float64
 3   reviewText_lemma                 63413 non-null  object 
 4   reviewText_nouns                 63413 non-null  object 
 5   reviewText_adjectives            63413 non-null  object 
 6   reviewText_verbs                 63413 non-null  object 
 7   reviewText_nav                   63413 non-null  object 
 8   reviewText_lemma_tb_pol          63310 non-null  float64
 9   reviewText_lemma_tb_subj         63310 non-null  float64
 10  reviewText_lemma_tb_tokens       63310 non-null  float64
 11  reviewText_lemma_tb_length       63310 non-null  float64
 12  reviewText_lemma_b

In [ ]:
#include positive negative mappings for analysis columns
#Note only stars of 4,5 are considered "positive"
dataOrig['overall_posneg'] = dataOrig.overall.apply(lambda x: 1 if x > 3 else 0)

In [ ]:
#import importlib
#from tqdm import tqdm, tqdm_pandas
#importlib.reload(cwutils)

In [ ]:
processColumns = ['reviewText_lemma_flairSent',
                  'reviewText_adjectives_flairSent',
                  'reviewText_verbs_flairSent',
                  'reviewText_nav_flairSent',
                  
                  'reviewText_lemma_tb_pol',
                  'reviewText_adjectives_tb_pol',
                  'reviewText_verbs_tb_pol',
                  'reviewText_nav_tb_pol'                  
                  ]

for col in processColumns:
  print(f'Processing column: {col}')
  dataOrig = cwutils.binPolarity(dataOrig, col)
  dataOrig = cwutils.binPolarityPosNeg(dataOrig, col)


Processing column: reviewText_lemma_flairSent


100%|██████████| 63413/63413 [00:00<00:00, 79469.37it/s] 


Processing column: reviewText_adjectives_flairSent


100%|██████████| 63413/63413 [00:00<00:00, 78828.47it/s] 


Processing column: reviewText_verbs_flairSent


...

100%|██████████| 63413/63413 [00:00<00:00, 75221.91it/s] 


Processing column: reviewText_verbs_tb_pol


100%|██████████| 63413/63413 [00:00<00:00, 102431.29it/s]


Processing column: reviewText_nav_tb_pol


100%|██████████| 63413/63413 [00:00<00:00, 91008.11it/s] 


In [ ]:
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 49 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   uuid                                    63413 non-null  object 
 1   reviewText                              63413 non-null  object 
 2   overall                                 63413 non-null  float64
 3   reviewText_lemma                        63413 non-null  object 
 4   reviewText_nouns                        63413 non-null  object 
 5   reviewText_adjectives                   63413 non-null  object 
 6   reviewText_verbs                        63413 non-null  object 
 7   reviewText_nav                          63413 non-null  object 
 8   reviewText_lemma_tb_pol                 63310 non-null  float64
 9   reviewText_lemma_tb_subj                63310 non-null  float64
 10  reviewText_lemma_tb_tokens              63310 non-null  fl

# Save Files!!!!!!!!

In [ ]:
dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/01_NL_ReviewText_All(new).pkl.gz')